### <p style="font-family: Arial; color: gold; font-weight: bold;">**create by Tom Tan in 8.30.2024** </p>
##### The idea is to create one notebook for each prefix. You only need to define the prefix in the follow cell and the notebook will do the rest.

***
# **1. Imports**

In [1]:
import pandas as pd

import get_properties_functions_for_WI as gp

***
# **2. Import the atom map from preprocess notebook**
### <p style="font-family: Arial; color: gold; font-weight: bold;"> **!!!User input required, Change the prefix to the one you want to handle.** </p>

In [2]:
prefix = "pyrz"

file_name = prefix + "_atom_map.xlsx"

atom_map_df = pd.read_excel(
    file_name, "Sheet1", index_col=0, header=0, engine="openpyxl"
)

display(atom_map_df.head())

df = atom_map_df  # df is what properties will be appended to, this creates a copy so that you have the original preserved

,log_name,C3,C4,N1,C5,C6,N2,C1,C2
0,pyrz1_conf-1,C2,C3,N5,C6,C11,N12,C4,C3
1,pyrz1_conf-2,C2,C3,N5,C6,C11,N12,C4,C3
2,pyrz2_conf-1,C2,C3,N5,C6,C11,N13,C4,C3
3,pyrz2_conf-2,C2,C3,N5,C6,C11,N13,C4,C3
4,pyrz3_conf-1,C2,C11,N10,C5,C4,N3,C1,C2


# **3. Define Properties to Collect**
### <p style="font-family: Arial; color: gold"> !!!User input required, Change/comment the properties block to the one you want to collect. </p>

In [3]:
# this box has functions to choose from
df = atom_map_df

# ---------------GoodVibes Engergies---------------
# uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
# calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
# inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

# ---------------Frontier Orbitals-----------------
# E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

# ---------------Polarizability--------------------
# Exact polarizability
df = gp.get_polarizability(df)

# ---------------Dipole----------------------------
# Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

# ---------------Volume----------------------------
# Molar volume
# requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

# ---------------SASA------------------------------
# Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

# ---------------NBO-------------------------------
# natural charge from NBO
# requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "C2", "N1", "N2"]
df = gp.get_nbo(df, nbo_list)

# ---------------NMR-------------------------------
# isotropic NMR shift
# requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C2", "N1", "N2"]
df = gp.get_nmr(df, nmr_list)

# ---------------Distance--------------------------
# distance between 2 atoms
dist_list_of_lists = [["C1", "C2"]]
df = gp.get_distance(df, dist_list_of_lists)

# ---------------Angle-----------------------------
# angle between 3 atoms
# angle_list_of_lists = [["C5", "N1", "C1"]]
# df = gp.get_angles(df, angle_list_of_lists)

# ---------------Dihedral--------------------------
# dihedral angle between 4 atoms
# dihedral_list_of_lists = [["C4", "C5", "N1", "C1"], ["C2", "C1", "N1", "C5"]]
# df = gp.get_dihedral(df, dihedral_list_of_lists)

# ---------------Vbur Scan-------------------------
# uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
# inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
# if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C2"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)

# ---------------Sterimol morfeus------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values
# NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["C1", "C2"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

# ---------------Buried Sterimol-------------------
# uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
# atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
# for more information: https://kjelljorner.github.io/morfeus/sterimol.html
# inputs: dataframe, list of atom pairs, r_buried
sterimol_list_of_lists = [["C1", "C2"]]
df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

# ---------------Sterimol DBSTEP-------------------
# uses DBSTEP to calculate Sterimol L, B1, and B5 values
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
# sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
# df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

# ---------------Sterimol2Vec----------------------
# uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
# default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
# more info here: https://github.com/patonlab/DBSTEP
# inputs: dataframe, list of atom pairs, end_radius, and step_size
# sterimol2vec_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
# df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

# ---------------Pyramidalization------------------
# uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
# collects values based on two definitions of pyramidalization
# details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["C1", "C2"]
df = gp.get_pyramidalization(df, pyr_list)

# ---------------Plane Angle-----------------------
# !plane angle between 2 planes (each defined by 6 atoms)
# planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
# df = gp.get_planeangle(df, planeangle_list_of_lists)

# --------------LP energy - custom from first cell---------------
lp_list = ["N1", "N2"]
df = gp.get_one_lp_energy(df, lp_list)

# ---------------Time----------------------------------
# returns the total CPU time and total Wall time (not per subjob) because we are pioneers
# if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
# df = gp.get_time(df)

# ---------------ChelpG----------------------------
# ChelpG ESP charge
# requires the Gaussian keyword = "pop=chelpg" in the .com file
# a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
# df = gp.get_chelpg(df, a_list)

# ---------------Hirshfeld-------------------------
# Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
# requires the Gaussian keyword = "pop=hirshfeld" in the .com file
# a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
# df = gp.get_hirshfeld(df, a_list)

pd.options.display.max_columns = None
display(df)

Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2', 'N1', 'N2']
NMR function has completed for ['C1', 'C2', 'N1', 'N2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 4
Morfeus Sterimol function has completed for [['C1', 'C2']]
Morfeus Buried Sterimol function has completed for [['C1', 'C2']]
Pyramidalization function has completed for ['C1', 'C2']
{'NBO_LP_occupancy_N1': '1.90484', 'NBO_LP_energy_N1': '-0.41674', 'NBO_LP_occupancy_N2': '1.90484', 'NBO_LP_energy_N2': '-0.41674'}
{'NBO_LP_occupancy_N1': '1.90443', 'NBO_LP_energy_N1': '-0.41727', 'NBO_LP_occupancy_N2': '1.90443', 'NBO_LP_energy_N2': '-0.41727'}
{'NBO_LP_occupancy_N1': '1.89884', 'NBO_LP_energy_N1': '-0.40818', 'NBO_LP_occupancy_N2': '1.89884', 'NBO_LP_energy_N2': '-0.40

,log_name,C3,C4,N1,C5,C6,N2,C1,C2,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NBO_charge_N1,NBO_charge_N2,NMR_shift_C1,NMR_shift_C2,NMR_shift_N1,NMR_shift_N2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,%Vbur_C1_2.5Å,%Vbur_C2_2.5Å,%Vbur_C1_3.0Å,%Vbur_C2_3.0Å,%Vbur_C1_3.5Å,%Vbur_C2_3.5Å,%Vbur_C1_4.0Å,%Vbur_C2_4.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,Buried_Sterimol_L_C1_C2_5.0(Å),Buried_Sterimol_B1_C1_C2_5.0(Å),Buried_Sterimol_B5_C1_C2_5.0(Å),pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,pyramidalization_Gavrish_C2(°),pyramidalization_Agranat-Radhakrishnan_C2,NBO_LP_occupancy_N1,NBO_LP_energy_N1,NBO_LP_occupancy_N2,NBO_LP_energy_N2
0,pyrz1_conf-1,C2,C3,N5,C6,C11,N12,C4,C3,-991.106148,0.137618,-990.956113,0.051041,0.049861,-991.007154,-991.005974,298.15,-0.32790,-0.05124,-0.189570,0.27666,0.06495,116.042,87.0864,2.5909,1376.671,355.909377,516.819811,0.875059,-0.64741,0.16626,-0.34408,-0.38741,160.2119,7.9563,-178.3072,-157.5203,1.49924,93.724174,95.703125,73.411251,86.231931,54.551549,74.456198,43.047661,61.897829,35.333002,49.441375,7.355359,1.980390,6.654961,6.789216,1.927656,5.056173,5.845762,0.793019,0.000000,9.161037e-12,1.90484,-0.41674,1.90484,-0.41674
1,pyrz1_conf-2,C2,C3,N5,C6,C11,N12,C4,C3,-991.104617,0.137518,-990.954629,0.051312,0.049998,-991.005941,-991.004627,298.15,-0.32878,-0.05100,-0.189890,0.27778,0.06490,116.184,88.9545,1.2306,1378.250,355.839370,516.744020,0.875146,-0.64811,0.16876,-0.33423,-0.38923,160.3884,6.2476,-188.8181,-155.4135,1.49918,93.708032,95.661157,73.411251,86.248209,54.545962,74.428263,43.031937,61.753983,35.240674,49.256331,8.754045,1.980711,5.421245,6.788573,1.928662,5.101110,5.839230,0.792343,0.000000,5.982543e-11,1.90443,-0.41727,1.90443,-0.41727
2,pyrz2_conf-1,C2,C3,N5,C6,C11,N13,C4,C3,-1046.483661,0.154530,-1046.315390,0.053439,0.052325,-1046.368829,-1046.367716,298.15,-0.28183,-0.04490,-0.163365,0.23693,0.05632,130.236,98.4587,1.3829,1561.342,369.799271,542.826229,0.870212,-0.63879,0.09892,-0.31653,-0.45767,162.3294,25.2775,-189.9457,-91.4368,1.50094,93.688662,95.686983,73.375440,86.144029,54.480781,74.307211,42.952735,61.903653,35.298088,49.536031,7.591408,1.981404,6.658801,7.536553,1.929370,5.068472,5.896118,0.798278,0.000028,4.258676e-06,1.89884,-0.40818,1.89884,-0.40818
3,pyrz2_conf-2,C2,C3,N5,C6,C11,N13,C4,C3,-1046.476561,0.154040,-1046.308449,0.054908,0.053226,-1046.363358,-1046.361675,298.15,-0.28333,-0.04349,-0.163410,0.23984,0.05567,129.747,100.2100,3.4464,1500.679,369.859219,543.668550,0.870971,-0.63964,0.10383,-0.30782,-0.45811,162.2502,22.8158,-203.8448,-92.5071,1.50083,93.759685,95.716038,73.336372,86.095195,54.459364,74.267171,42.916045,61.815132,35.211967,49.401806,8.582079,1.977304,5.658049,7.541063,1.928687,5.125942,5.885690,0.797209,0.009708,1.458358e-03,1.89873,-0.40910,1.89873,-0.40910
4,pyrz3_conf-1,C2,C11,N10,C5,C4,N3,C1,C2,-531.500798,0.147156,-531.342230,0.048636,0.047251,-531.390866,-531.389481,298.15,-0.32334,-0.04286,-0.183100,0.28048,0.05976,102.117,74.4940,2.1612,1084.711,332.949610,474.464778,0.883571,-0.64343,0.19083,-0.33472,-0.39349,162.2300,0.9057,-177.6629,-172.5223,1.50139,93.001033,95.735408,71.114403,85.590572,51.443310,72.132002,39.429393,57.049595,31.068835,42.430250,9.802480,1.984219,3.256895,7.779822,1.886801,3.254607,5.941155,0.802926,0.000077,1.151187e-05,1.92347,-0.40547,1.92347,-0.40547
5,pyrz3_conf-2,C2,C11,N10,C5,C4,N3,C1,C2,-531.499018,0.147043,-531.340507,0.048981,0.047410,-531.389488,-531.387917,298.15,-0.32533,-0.04275,-0.184040,0.28258,0.05993,102.103,75.0045,2.2492,1163.566,333.139613,474.820761,0.883509,-0.64365,0.19148,-0.32417,-0.39506,162.3250,1.6090,-189.4702,-170.2931,1.50137,93.013946,95.732180,71.114403,85.598711,51.409788,72.10

## 3.1 Save collected properties to Excel and pickle file

In [4]:
# save the pandas dataframe to a xlsx file
with pd.ExcelWriter(prefix + "_extracted_properties.xlsx") as writer:
    df.to_excel(writer)
# save the pandas dataframe to a pickle file
df.to_pickle(prefix + "_extracted_properties.pkl")

# **4. Post-processing**

In [5]:
import re
import pandas as pd
import numpy as np
from tabulate import tabulate

In [6]:
# for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
# this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
prefix = "pyrz"
suffix = "_"

# columns that provide atom mapping information are dropped
atom_columns_to_drop = ["C1", "C2", "C3", "C4", "N1", "N2"]

# title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

##### If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

In [7]:
df = pd.read_excel(
    prefix + "_extracted_properties.xlsx",
    "Sheet1",
    index_col=0,
    header=0,
    engine="openpyxl",
)
display(df.head())

,log_name,C3,C4,N1,C5,C6,N2,C1,C2,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NBO_charge_N1,NBO_charge_N2,NMR_shift_C1,NMR_shift_C2,NMR_shift_N1,NMR_shift_N2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,%Vbur_C1_2.5Å,%Vbur_C2_2.5Å,%Vbur_C1_3.0Å,%Vbur_C2_3.0Å,%Vbur_C1_3.5Å,%Vbur_C2_3.5Å,%Vbur_C1_4.0Å,%Vbur_C2_4.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,Buried_Sterimol_L_C1_C2_5.0(Å),Buried_Sterimol_B1_C1_C2_5.0(Å),Buried_Sterimol_B5_C1_C2_5.0(Å),pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,pyramidalization_Gavrish_C2(°),pyramidalization_Agranat-Radhakrishnan_C2,NBO_LP_occupancy_N1,NBO_LP_energy_N1,NBO_LP_occupancy_N2,NBO_LP_energy_N2
0,pyrz1_conf-1,C2,C3,N5,C6,C11,N12,C4,C3,-991.106148,0.137618,-990.956113,0.051041,0.049861,-991.007154,-991.005974,298.15,-0.32790,-0.05124,-0.189570,0.27666,0.06495,116.042,87.0864,2.5909,1376.671,355.909377,516.819811,0.875059,-0.64741,0.16626,-0.34408,-0.38741,160.2119,7.9563,-178.3072,-157.5203,1.49924,93.724174,95.703125,73.411251,86.231931,54.551549,74.456198,43.047661,61.897829,35.333002,49.441375,7.355359,1.980390,6.654961,6.789216,1.927656,5.056173,5.845762,0.793019,0.000000,9.161037e-12,1.90484,-0.41674,1.90484,-0.41674
1,pyrz1_conf-2,C2,C3,N5,C6,C11,N12,C4,C3,-991.104617,0.137518,-990.954629,0.051312,0.049998,-991.005941,-991.004627,298.15,-0.32878,-0.05100,-0.189890,0.27778,0.06490,116.184,88.9545,1.2306,1378.250,355.839370,516.744020,0.875146,-0.64811,0.16876,-0.33423,-0.38923,160.3884,6.2476,-188.8181,-155.4135,1.49918,93.708032,95.661157,73.411251,86.248209,54.545962,74.428263,43.031937,61.753983,35.240674,49.256331,8.754045,1.980711,5.421245,6.788573,1.928662,5.101110,5.839230,0.792343,0.000000,5.982543e-11,1.90443,-0.41727,1.90443,-0.41727
2,pyrz2_conf-1,C2,C3,N5,C6,C11,N13,C4,C3,-1046.483661,0.154530,-1046.315390,0.053439,0.052325,-1046.368829,-1046.367716,298.15,-0.28183,-0.04490,-0.163365,0.23693,0.05632,130.236,98.4587,1.3829,1561.342,369.799271,542.826229,0.870212,-0.63879,0.09892,-0.31653,-0.45767,162.3294,25.2775,-189.9457,-91.4368,1.50094,93.688662,95.686983,73.375440,86.144029,54.480781,74.307211,42.952735,61.903653,35.298088,49.536031,7.591408,1.981404,6.658801,7.536553,1.929370,5.068472,5.896118,0.798278,0.000028,4.258676e-06,1.89884,-0.40818,1.89884,-0.40818
3,pyrz2_conf-2,C2,C3,N5,C6,C11,N13,C4,C3,-1046.476561,0.154040,-1046.308449,0.054908,0.053226,-1046.363358,-1046.361675,298.15,-0.28333,-0.04349,-0.163410,0.23984,0.05567,129.747,100.2100,3.4464,1500.679,369.859219,543.668550,0.870971,-0.63964,0.10383,-0.30782,-0.45811,162.2502,22.8158,-203.8448,-92.5071,1.50083,93.759685,95.716038,73.336372,86.095195,54.459364,74.267171,42.916045,61.815132,35.211967,49.401806,8.582079,1.977304,5.658049,7.541063,1.928687,5.125942,5.885690,0.797209,0.009708,1.458358e-03,1.89873,-0.40910,1.89873,-0.40910
4,pyrz3_conf-1,C2,C11,N10,C5,C4,N3,C1,C2,-531.500798,0.147156,-531.342230,0.048636,0.047251,-531.390866,-531.389481,298.15,-0.32334,-0.04286,-0.183100,0.28048,0.05976,102.117,74.4940,2.1612,1084.711,332.949610,474.464778,0.883571,-0.64343,0.19083,-0.33472,-0.39349,162.2300,0.9057,-177.6629,-172.5223,1.50139,93.001033,95.735408,71.114403,85.590572,51.443310,72.132002,39.429393,57.049595,31.068835,42.430250,9.802480,1.984219,3.256895,7.779822,1.886801,3.254607,5.941155,0.802926,0.000077,1.151187e-05,1.92347,-0.40547,1.92347,-0.40547


## 4.1 Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [8]:
compound_list = []

for index, row in df.iterrows():
    log_file = row["log_name"]  # read file name from df
    prefix_and_compound = log_file.split(
        str(suffix)
    )  # splits to get "AcXXX" (entry O) (and we don't use the "clust-X" (entry 1))
    compound = prefix_and_compound[0].split(
        str(prefix)
    )  # splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
    compound_list.append(compound[1])

compound_list = list(
    set(compound_list)
)  # removes duplicate stuctures that result from having conformers of each
compound_list.sort(
    key=lambda x: int(re.search(r"\d+", x).group())
)  # reorders numerically (not sure if it reorders alphabetically)
print(compound_list)

# this should generate a list that looks like this: ['24', '27', '34', '48']

['1', '2', '3']


## 4.2 Post-processing to get properties for each compound

##### changes made in 8/30/2024 <br> 1. avoid divide by zero error in the Boltzmann averaging, the original code had the if block order reversed, which caused the error. <br> 2. data cleaning by remove columns contain cell that cannot be converted to float. <br> 3. concat all data into row before concat them into the final dataframe. The originl modify individual cells which result in fragmented and raise performance warning.

In [9]:
all_df_master = pd.DataFrame(columns=[])
properties_df_master = pd.DataFrame(columns=[])

for compound in compound_list:
    # defines the common start to all files using the input above
    substring = str(prefix) + str(compound) + str(suffix)

    # makes a data frame for one compound at a time for post-processing
    valuesdf = df[df["log_name"].str.startswith(substring)]
    valuesdf = valuesdf.drop(columns=atom_columns_to_drop)
    valuesdf = valuesdf.reset_index(
        drop=True
    )  # you must re-index otherwise the 2nd, 3rd, etc. compounds fail

    # filter column that are characters, we will attempt to convert them to numbers, if fail, we will drop them
    for column in valuesdf:
        try:
            # exclude column "log_name"
            if column == "log_name":
                continue
            valuesdf[column] = pd.to_numeric(valuesdf[column])
        except:
            valuesdf = valuesdf.drop(columns=column)
            valuesdf = valuesdf.reset_index(
                drop=True
            )  # reset the index after dropping columns

    # define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
    non_boltz_columns = [
        "G(Hartree)",
        "∆G(Hartree)",
        "∆G(kcal/mol)",
        "e^(-∆G/RT)",
        "Mole Fraction",
    ]  # don't boltzman average columns containing these strings in the column label
    reg_avg_columns = [
        "CPU_time_total(hours)",
        "Wall_time_total(hours)",
    ]  # don't boltzmann average these either, we average them in case that is helpful
    gv_extra_columns = [
        "E_spc (Hartree)",
        "H_spc(Hartree)",
        "T",
        "T*S",
        "T*qh_S",
        "ZPE(Hartree)",
        "qh_G(T)_spc(Hartree)",
        "G(T)_spc(Hartree)",
    ]
    gv_extra_columns.remove(str(energy_col_header))

    # calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
    valuesdf["∆G(Hartree)"] = (
        valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
    )
    valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
    valuesdf["e^(-∆G/RT)"] = np.exp(
        (valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)
    )  # R is in cal/(K*mol)
    valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
    values_boltz_row = []
    values_min_row = []
    values_max_row = []
    values_boltz_stdev_row = []
    values_range_row = []
    values_exclude_columns = []

    for column in valuesdf:
        if "log_name" in column:
            values_boltz_row.append("Boltzmann Averages")
            values_min_row.append("Ensemble Minimum")
            values_max_row.append("Ensemble Maximum")
            values_boltz_stdev_row.append("Boltzmann Standard Deviation")
            values_range_row.append("Ensemble Range")
            values_exclude_columns.append(column)  # used later to build final dataframe
        elif any(phrase in column for phrase in non_boltz_columns) or any(
            phrase in column for phrase in gv_extra_columns
        ):
            values_boltz_row.append("")
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        elif any(phrase in column for phrase in reg_avg_columns):
            values_boltz_row.append(
                valuesdf[column].mean()
            )  # intended to print the average CPU/wall time in the boltz column
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        else:
            valuesdf[column] = pd.to_numeric(
                valuesdf[column]
            )  # to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
            values_boltz_row.append(
                (valuesdf[column] * valuesdf["Mole Fraction"]).sum()
            )
            values_min_row.append(valuesdf[column].min())
            values_max_row.append(valuesdf[column].max())
            values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

            # this section generates the weighted std deviation (weighted by mole fraction)
            # formula: https://www.statology.org/weighted-standard-deviation-excel/

            boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum()  # number
            delta_values_sq = []

            # makes a list of the "deviation" for each conformer
            for index, row in valuesdf.iterrows():
                value = row[column]
                delta_value_sq = (value - boltz) ** 2
                delta_values_sq.append(delta_value_sq)

            # w is list of weights (i.e. mole fractions)
            w = list(valuesdf["Mole Fraction"])
            # !swap the order here to avoid division by zero error
            if (
                len(w) == 1
            ):  # if there is only one conformer in the ensemble, set the weighted standard deviation to 0
                wstdev = 0
            # np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
            else:
                wstdev = np.sqrt(
                    (np.average(delta_values_sq, weights=w))
                    / (((len(w) - 1) / len(w)) * np.sum(w))
                )
            values_boltz_stdev_row.append(wstdev)

    valuesdf.loc[len(valuesdf)] = values_boltz_row
    valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
    valuesdf.loc[len(valuesdf)] = values_min_row
    valuesdf.loc[len(valuesdf)] = values_max_row
    valuesdf.loc[len(valuesdf)] = values_range_row

    # final output format is built here:
    explicit_order_front_columns = [
        "log_name",
        energy_col_header,
        "∆G(Hartree)",
        "∆G(kcal/mol)",
        "e^(-∆G/RT)",
        "Mole Fraction",
    ]

    # reorders the dataframe using front columns defined above
    valuesdf = valuesdf[
        explicit_order_front_columns
        + [
            col
            for col in valuesdf.columns
            if col not in explicit_order_front_columns
            and col not in values_exclude_columns
        ]
    ]

    # determine the index of the lowest energy conformer
    low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
    # copy the row to a new_row with the name of the log changed to Lowest E Conformer
    new_row = pd.DataFrame(valuesdf.loc[low_e_index[0]]).T
    new_row["log_name"] = "Lowest E Conformer"
    # check if there is empty/Nan values in the row
    if new_row.isnull().values.any():
        print("There are empty values in the row")
        # print the entire row to see where the empty values are
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
        # drop the columns with empty values
        new_row = new_row.dropna(axis=1, how="any")
        print("The empty values have been dropped")
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
    valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

    # ------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------
    # if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
    # this template can be adjusted for min/max/etc.

    # find the index for the min or max column:
    ensemble_min_index = valuesdf[
        valuesdf["log_name"] == "Ensemble Minimum"
    ].index.tolist()
    # find the min or max value of the property (based on index above)
    # saves the value in a list (min_value) with one entry (this is why we call min_value[0])
    min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C1_3.0Å"].tolist()
    vbur_min_index = valuesdf[valuesdf["%Vbur_C1_3.0Å"] == min_value[0]].index.tolist()
    # copy the row to a new_row with the name of the log changed to Property_min_conformer
    new_row = pd.DataFrame(valuesdf.loc[vbur_min_index[0]]).T
    new_row["log_name"] = "%Vbur_C1_3.0Å_min_Conformer"
    # check if there is empty/Nan values in the row
    if new_row.isnull().values.any():
        print("There are empty values in the row")
        # print the entire row to see where the empty values are
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
        # drop the columns with empty values
        new_row = new_row.dropna(axis=1, how="any")
        print("The empty values have been dropped")
        print(tabulate(new_row, headers="keys", tablefmt="pretty"))
    valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)
    # --------------------------------------------------------------------------------------------------------------------

    # appends the frame to the master output
    all_df_master = pd.concat([all_df_master, valuesdf])

    # drop all the individual conformers
    dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
    valuesdf = valuesdf.drop(dropindex)
    valuesdf = valuesdf.reset_index(drop=True)

    # drop the columns created to determine the mole fraction and some that
    valuesdf = valuesdf.drop(columns=explicit_order_front_columns)
    try:
        valuesdf = valuesdf.drop(columns=gv_extra_columns)
    except:
        pass
    try:
        valuesdf = valuesdf.drop(columns=reg_avg_columns)
    except:
        pass

    # ---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------
    compound_name = prefix + str(compound)
    # --------------------------------------------------------------------------------------------------------------------

    properties_df = pd.DataFrame({"Compound_Name": [compound_name]})

    # builds a dataframe (for each compound) by adding summary properties as new columns
    for column in valuesdf:
        # the indexes need to match the values dataframe - display it to double check if you need to make changes
        # (uncomment the display(valuesdf) in row 124 of this cell)

        # create a list of headers for the properties_df
        # if you're collecting properties for a specific conformer, edit the header to reflect that, it should match the order in the valuesdf log_name column
        headers = [
            f"{column}_Boltz",
            f"{column}_Boltz_stdev",
            f"{column}_min",
            f"{column}_max",
            f"{column}_range",
            f"{column}_low_E",
            f"{column}_Vbur_min",
        ]
        # Extract values for the current column from valuesdf and create a DataFrame
        row_dataframe = pd.DataFrame([valuesdf[column].values], columns=headers)
        # Display the DataFrame for verification
        # display(row_dataframe)
        # Concatenate the new DataFrame to the properties_df along the columns (axis=1)
        properties_df = pd.concat([properties_df, row_dataframe], axis=1)

    # concatenates the individual acid properties df into the master properties df
    properties_df_master = pd.concat([properties_df_master, properties_df], axis=0)

# Reset the index of the master DataFrames
all_df_master = all_df_master.reset_index(drop=True)
properties_df_master = properties_df_master.reset_index(drop=True)

# 5. Export the data

In [10]:
# Print in tabulated version
print(tabulate(properties_df_master, headers="keys", tablefmt="pretty"))
print(tabulate(all_df_master, headers="keys", tablefmt="pretty"))

# grep the first & last Compound_Name
first_compound = str(properties_df_master["Compound_Name"].iloc[0])
last_compound = str(properties_df_master["Compound_Name"].iloc[-1])

# Define the filename for the Excel file
filename = (
    prefix
    + "_properties_postprocessed_"
    + "for_"
    + first_compound
    + "_to_"
    + last_compound
    + ".xlsx"
)

# export to excel
with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
    all_df_master.to_excel(writer, sheet_name="All_Conformer_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in all_df_master.columns:
        column_width = max(
            all_df_master[column].astype(str).map(len).max(), len(column)
        )
        col_idx = all_df_master.columns.get_loc(column)
        writer.sheets["All_Conformer_Properties"].set_column(
            col_idx, col_idx, column_width
        )
    properties_df_master.to_excel(writer, sheet_name="Summary_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in properties_df_master.columns:
        column_width = max(
            properties_df_master[column].astype(str).map(len).max(), len(column)
        )
        col_idx = properties_df_master.columns.get_loc(column)
        writer.sheets["Summary_Properties"].set_column(col_idx, col_idx, column_width)

+---+---------------+---------------------+------------------------+----------+----------+-----------------------+------------+---------------+----------------------+------------------------+----------+----------+------------------------+------------+---------------+----------------------+------------------------+----------+-----------+-----------------------+-----------+------------+---------------------+------------------------+---------+---------+-----------------------+---------+------------+----------------------+-----------------------+---------+---------+------------------------+---------+------------+------------------------+------------------------------+----------------------+----------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------------+------------------------+------------------------+--------------------------+--------------------------+-----------------------------+----------

In [11]:
# Define filenames for the pickle files
pkl_filename_all = (
    prefix
    + "_properties_postprocessed"
    + "_all_conformer_properties"
    + "_for_"
    + prefix
    + ".pkl"
)
pkl_filename_summary = (
    prefix
    + "_properties_postprocessed"
    + "_summary_properties"
    + "_for_"
    + prefix
    + ".pkl"
)

# Save to pickle
all_df_master.to_pickle(pkl_filename_all)
properties_df_master.to_pickle(pkl_filename_summary)